<a href="https://colab.research.google.com/github/hadefpetriza/C23-PC679/blob/main/Project_Capstone_C23_PC679.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification
**Note :**
Jadi Aplikasi dapat mendeteksi kelima item berikut
1.   Sampah Organic
2.   Plastik (Recycle)
3.   Kayu (Recycle)
4.   Seng / besi (Recycle)
5.   Non-Recycle(Sampah medis, sampah basah, sampah kaca)



In [ ]:
# Library yang digunakan
import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
# import tensorboard
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# File path for data
train_dataset = "????"
val_dataset = "????"
test_dataset = "????"

In [ ]:
# Show the data in train_dataset
jenis = ["Sampah Organic", "Sampah Plastik", "Sampah Kayu", "Sampah Besi", "Sampah Non-Recycle"]
plt.figure(figsize=(10,10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(jenis[labels[i]])
    plt.axis("off")

In [ ]:
# Image Augmentation 
def train_val_generator(train_dataset, val_dataset):
 train_datagen = ImageDataGenerator(rescale = 1./255.,
                                     rotation_range = 40,
                                     width_shift_range = 0.2,
                                     height_shift_range = 0.2,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     fill_mode = 'nearest')
 
  train_generator = train_datagen.flow_from_directory(directory = train_dataset,
                                                      batch_size = 250,
                                                      class_mode = 'categorical',
                                                      target_size = (224, 224))

  val_datagen = ImageDataGenerator(rescale = 1./255.)
  val_generator = val_datagen.flow_from_directory(directory = val_dataset,
                                                                batch_size = 250,
                                                                class_mode = 'categorical',
                                                                target_size = (150, 150))
  
  return train_generator, val_generator

In [ ]:
# Model Architecture
def create_model():
  model = tf.keras.Sequential[(
      tf.keras.layers.Conv2D(64, (3, 3), activation = "relu", input_shape = (300, 300, 3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation = "relu"),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(activation = "relu"),
      tf.keras.layers.MaxPooling2D(2,2),
      
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation = "relu"),
      tf.keras.layers.Dense(5, activation =  "softmax")
  )]

  model.compile(optimizer = tf.optimizers.Adam(),
                loss = "sparse_categorical_crossentropy",
                metrics = "accuracy")
  
  return model



In [ ]:
# Save model
model = create_model()

# Train Model
history = model.fit(train_generator,
                    epoch = 20,
                    validation_data = val_generator)

In [ ]:
# Show plot train accuracy and val_accuracy
plt.plot(history.history["accuracy"], label = "accuracy")
plt.plot(history.history["val_accuracy"], label = "val_accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([0.5, 1])
plt.legend(loc = "lower right")

In [ ]:
# Save file training (TensorFlow Lite)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model_classification.tflite", 'wb') as f:
  f.write(tflite_model)